# Chapter 9: Sparse Neural Networks

Forward propagation through sparse weight matrices using GraphBLAS.

In [ ]:
import graphblas as gb
from graphblas import Matrix, Vector, semiring, binary
import numpy as np
import matplotlib.pyplot as plt

## Dense vs Sparse Weights

Many neural network weights are near-zero and can be pruned.

In [ ]:
# Simulated dense weights with many small values
np.random.seed(42)
dense_weights = np.random.randn(6, 4) * 0.5
dense_weights[np.abs(dense_weights) < 0.3] = 0  # Prune small weights

print("Dense weight matrix (zeros shown):")
print(np.round(dense_weights, 2))
print(f"\nNon-zero elements: {np.count_nonzero(dense_weights)} / {dense_weights.size}")

In [ ]:
# Convert to sparse GraphBLAS matrix
rows, cols = np.nonzero(dense_weights)
vals = dense_weights[rows, cols]

W = Matrix.from_coo(rows.tolist(), cols.tolist(), vals.tolist(), 
                    nrows=6, ncols=4, dtype=float)
print("Sparse weight matrix:")
print(W)

## Forward Pass: x × W

In [ ]:
# Input vector (sparse: only some features active)
x = Vector.from_coo([0, 2, 5], [1.0, 0.5, -0.8], size=6, dtype=float)
print("Sparse input x:")
print(x)

In [ ]:
# Forward pass: y = x × W (vector-matrix multiply)
y = x.vxm(W, semiring.plus_times).new()
print("Output y = x × W:")
print(y)

## Activation Functions

In [ ]:
# ReLU: max(0, x)
def relu(v):
    return v.select(">", 0).new()

y_relu = relu(y)
print("After ReLU:")
print(y_relu)

In [ ]:
# Threshold activation (binary)
def threshold(v, t=0.0):
    return v.select(">", t).apply(binary.pair).new(dtype=bool)

y_thresh = threshold(y, 0.0)
print("After threshold (> 0):")
print(y_thresh)

## Multi-Layer Network

In [ ]:
# Create a 2-layer network: 6 -> 4 -> 2
np.random.seed(123)

# Layer 1: 6 inputs -> 4 hidden
w1_dense = np.random.randn(6, 4) * 0.5
w1_dense[np.abs(w1_dense) < 0.4] = 0
r, c = np.nonzero(w1_dense)
W1 = Matrix.from_coo(r.tolist(), c.tolist(), w1_dense[r, c].tolist(),
                     nrows=6, ncols=4, dtype=float)

# Layer 2: 4 hidden -> 2 outputs
w2_dense = np.random.randn(4, 2) * 0.5
w2_dense[np.abs(w2_dense) < 0.3] = 0
r, c = np.nonzero(w2_dense)
W2 = Matrix.from_coo(r.tolist(), c.tolist(), w2_dense[r, c].tolist(),
                     nrows=4, ncols=2, dtype=float)

print("W1 (6x4):")
print(W1)
print("\nW2 (4x2):")
print(W2)

In [ ]:
# Forward pass through both layers
x = Vector.from_coo([0, 3, 5], [1.0, -0.5, 0.8], size=6, dtype=float)
print("Input:")
print(x)

# Layer 1 + ReLU
h = relu(x.vxm(W1, semiring.plus_times).new())
print("\nHidden (after ReLU):")
print(h)

# Layer 2
output = h.vxm(W2, semiring.plus_times).new()
print("\nOutput:")
print(output)

## Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Dense weights
axes[0].imshow(np.abs(w1_dense), cmap='Blues')
axes[0].set_title('Layer 1 Weights (magnitude)')
axes[0].set_xlabel('Output neurons')
axes[0].set_ylabel('Input neurons')

# Sparse pattern
sparse_pattern = (np.abs(w1_dense) > 0).astype(int)
axes[1].imshow(sparse_pattern, cmap='binary')
axes[1].set_title('Sparsity Pattern (black = non-zero)')
axes[1].set_xlabel('Output neurons')
axes[1].set_ylabel('Input neurons')

plt.tight_layout()
plt.show()

sparsity = 1 - np.count_nonzero(w1_dense) / w1_dense.size
print(f"Sparsity: {sparsity:.1%}")